##Loading pre trained model and then using our drawing recognizing the digits


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from PIL import Image
import io

# Load the pre-trained MNIST model from Keras
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Load MNIST dataset and train the model
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

model.fit(x_train, y_train, epochs=15, validation_data=(x_test, y_test))

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 49s 24ms/step - accuracy: 0.9142 - loss: 0.2913 - val_accuracy: 0.9805 - val_loss: 0.0616
Epoch 2/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 40s 21ms/step - accuracy: 0.9849 - loss: 0.0506 - val_accuracy: 0.9845 - val_loss: 0.0452
Epoch 3/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 21ms/step - accuracy: 0.9902 - loss: 0.0316 - val_accuracy: 0.9853 - val_loss: 0.0437
Epoch 4/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 40s 21ms/step - accuracy: 0.9942 - loss: 0.0188 - val_accuracy: 0.9871 - val_loss: 0.0425
Epoch 5/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9961 - loss: 0.0127 - val_accuracy: 0.9845 - val_loss: 0.0498
Epoch 6/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 21ms/step - accuracy: 0.9967 - loss: 0.0096 - val_accuracy: 0.9860 - val_loss: 0.0523
Epoch 7/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accuracy: 0.9975 - loss: 0.0070 - val_accuracy: 0.9879 - val_loss: 0.0449
Epoch 8/15
1875/1875 ━━━━

In [ ]:
# Function to display the drawing interface
def draw():
    display(Javascript('''
      const canvas = document.createElement('canvas');
      canvas.width = 200;
      canvas.height = 200;
      const ctx = canvas.getContext('2d');
      ctx.fillStyle = 'white';
      ctx.fillRect(0, 0, canvas.width, canvas.height);

      let drawing = false;

      canvas.addEventListener('mousedown', (e) => {
        drawing = true;
        ctx.moveTo(e.clientX - canvas.offsetLeft, e.clientY - canvas.offsetTop);
      });

      canvas.addEventListener('mousemove', (e) => {
        if (drawing) {
          ctx.lineTo(e.clientX - canvas.offsetLeft, e.clientY - canvas.offsetTop);
          ctx.stroke();
        }
      });

      canvas.addEventListener('mouseup', () => {
        drawing = false;
      });

      document.body.appendChild(canvas);

      const saveButton = document.createElement('button');
      saveButton.textContent = 'Predict';
      saveButton.onclick = () => {
        const dataURL = canvas.toDataURL('image/png');
        google.colab.kernel.invokeFunction('notebook.getDrawing', [dataURL], {});
      };
      document.body.appendChild(saveButton);
    '''))

# Function to process and predict
def get_drawing(data_url):
    image_bytes = b64decode(data_url.split(',')[1])
    image = Image.open(io.BytesIO(image_bytes)).convert('L')
    image = image.resize((28, 28))  # Resize to 28x28 pixels
    image = np.array(image)
    image = image.reshape(1, 28, 28, 1).astype('float32') / 255.0
    prediction = model.predict(image)
    plt.imshow(image.reshape(28, 28), cmap='gray')
    plt.title(f'Predicted Digit: {np.argmax(prediction)}')
    plt.show()

# Register function to be called from JS
from google.colab import output
output.register_callback('notebook.getDrawing', get_drawing)

# Display the drawing interface
draw()

NameError: name 'Javascript' is not defined